In [1]:
#!pip install PyNaver

In [128]:
from PyNaver import Datalab
import pandas as pd
import urllib.request
from datetime import datetime, timedelta
import json
import sys
import os
import warnings
import re
warnings.filterwarnings(action='ignore')

today = str(datetime.now().date())
recent_8_days = str(datetime.now().date() - timedelta(weeks=1,days=1))
regex = '\(.*\)|\s-\s'

In [233]:
ratio_df = pd.read_csv('sub_day_search.csv')
recent_search = pd.read_csv('recent_search.csv')

In [219]:
days = pd.read_csv('38_day.csv',encoding='euc-kr')
days = days[~days['기업명'].str.contains('스팩')]
days.drop(['Unnamed: 0'], axis = 1,inplace = True)
days["청약일"] = days["청약일"].str.replace(pat=r'[^\w]', repl=r'', regex=True)
days.drop(['수요예측일'], axis = 1,inplace = True)
days['청약일'] = pd.to_datetime(days['청약일'])
days = days.reset_index()
days.drop(['index'],axis=1,inplace=True)
for i in range(len(days['기업명'])):
    days['기업명'][i] = re.sub(regex,'',days['기업명'][i])

In [220]:
class NaverDataLabOpenAPI():
    """
    네이버 데이터랩 오픈 API 컨트롤러 클래스
    """
    def __init__(self, client_id, client_secret):
        """
        인증키 설정 및 검색어 그룹 초기화
        """
        self.client_id = client_id
        self.client_secret = client_secret
        self.keywordGroups = []
        self.url = "https://openapi.naver.com/v1/datalab/search"
    def add_keyword_groups(self, group_dict):
        """
        검색어 그룹 추가
        """
        keyword_gorup = {
            'groupName': group_dict['groupName'],
            'keywords': group_dict['keywords']
        }
        
        self.keywordGroups.append(keyword_gorup)
        #print(f">>> Num of keywordGroups: {len(self.keywordGroups)}")
        
    def get_data(self, startDate, endDate, timeUnit, device, ages, gender):
        """
        요청 결과 반환
        timeUnit - 'date', 'week', 'month'
        device - None, 'pc', 'mo'
        ages = [], ['1' ~ '11']
        gender = None, 'm', 'f'
        """
        # Request body
        body = json.dumps({
            "startDate": startDate,
            "endDate": endDate,
            "timeUnit": timeUnit,
            "keywordGroups": self.keywordGroups,
            "device": device,
            "ages": ages,
            "gender": gender
        }, ensure_ascii=False)
        
        # Results
        request = urllib.request.Request(self.url)
        request.add_header("X-Naver-Client-Id",self.client_id)
        request.add_header("X-Naver-Client-Secret",self.client_secret)
        request.add_header("Content-Type","application/json")
        response = urllib.request.urlopen(request, data=body.encode("utf-8"))
        rescode = response.getcode()
        if(rescode==200):
            # Json Result
            result = json.loads(response.read())
            
            df = pd.DataFrame(result['results'][0]['data'])[['period']]
            for i in range(len(self.keywordGroups)):
                #df['period'] = df.apply(lambda r: [df['period'].values], axis=1, result_type='expand')
                tmp = pd.DataFrame(result['results'][i]['data'])
                tmp = tmp.rename(columns={'ratio': result['results'][i]['title']})
                df = pd.merge(df, tmp, how='left', on=['period'])
            self.df = df.rename(columns={'period': '날짜'})
            self.df['날짜'] = pd.to_datetime(self.df['날짜'])
            
        else:
            print("Error Code:" + rescode)
            
        return self.df

In [ ]:
#상장일 2일동안의 검색량 
today = str(datetime.now().date())
recent_8_days = str(datetime.now().date() - timedelta(weeks=1,days=1))
# API 인증 정보 설정
client_id = "CN4hF3ElCR7yM8WvZ3zI"
client_secret = "7ACegbbksF"

for i in range(430,493):
    if days['기업명'][i] not in ratio_df['기업명']:
        name = days['기업명'][i]
        sub_date = str(days['청약일'][i].date())
        day_after = str(days['청약일'][i].date() + timedelta(days=1))
    else:
        print('크롤링완료')
        break
        
    keyword_group_set = {
    'keyword_group_1': {'groupName': '검색량', 'keywords': [name]}
                    }
    # 요청 파라미터 설정
    startDate = sub_date
    endDate = day_after
    timeUnit = 'date'
    device = ''
    ages = []
    gender = ''

# 데이터 프레임 정의
    naver = NaverDataLabOpenAPI(client_id=client_id, client_secret=client_secret)

    naver.add_keyword_groups(keyword_group_set['keyword_group_1'])

    df = naver.get_data(startDate, endDate, timeUnit, device, ages, gender)
    df['기업명'] = name
    ratio_df = pd.concat([ratio_df,df])

In [140]:
#최근 8일동안 검색량 
client_id = '5WCkVtSTuUn6tGDdxwfd'
client_secret = 'EHuFif9z6h'
for i in range(len(days['청약일'])):
    if days['기업명'][i] not in recent_search['기업명'].values:
        name = days['기업명'][i]
    else:
        continue 
    url = "https://openapi.naver.com/v1/datalab/search"
    body = "{\"startDate\":\""+recent_8_days+"\",\"endDate\":\""+today+"\",\"timeUnit\":\"date\",\"keywordGroups\":[{\"groupName\":\""+name+"\",\"keywords\":[\""+name+"\"]}]}";
    requested = urllib.request.Request(url)
    requested.add_header("X-Naver-Client-Id", client_id)
    requested.add_header("X-Naver-Client-Secret", client_secret)
    requested.add_header("Content-Type", "application/json")
    response = urllib.request.urlopen(requested, data=body.encode("utf-8"))
    rescode = response.getcode()

    if(rescode==200):
        response_body = response.read()
        output_data = response_body.decode('utf-8')
    else:
        print('Error code:'+ rescode)
        continue


    result = json.loads(output_data)
    
    date = [a['period'] for a in result['results'][0]['data']]
    corp_ratio = [a['ratio'] for a in result['results'][0]['data']]

    ratio = pd.DataFrame({'date':date, 
                  '기업검색량':corp_ratio,
                '기업명':name})
    recent_search = pd.concat([recent_search,ratio])
recent_search

In [231]:
ratio_df = ratio_df.to_csv('sub_day_search.csv',index=False)
recent_search = recent_search.to_csv('recent_search.csv', index=False)

AttributeError: 'NoneType' object has no attribute 'to_csv'

In [63]:
#429: 코썬바이오 오류

In [244]:
sub_day_search = ratio_df.groupby('기업명').mean()
sub_day_search.drop(['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1'],axis=1,inplace=True)
sub_day_search.rename(columns={'검색량':'청약일검색량'},inplace=True)
sub_day_search

,청약일검색량
기업명,
ING생명보험,57.449205
LG에너지솔루션,85.464685
NH올원리츠,57.859765
SK리츠,63.899235
SK바이오사이언스,74.984995
...,...
현대중공업,73.482235
호전실업,60.733840
화승엔터프라이즈,61.744120


In [253]:
recent_search = recent_search.groupby('기업명').median()
#recent_search.drop(['Unnamed: 0'],axis=1,inplace=True)
recent_search.rename(columns={'기업검색량':'최근일주일검색량'},inplace=True)
recent_search

,최근일주일검색량
기업명,
BGF리테일,65.832510
CJ헬로비전,73.512959
GKL,67.037464
GMB코리아,53.988863
GS리테일,48.817154
...,...
휴비스,78.823289
휴젤,68.569273
흥국,72.959179


In [258]:
merged = pd.merge(sub_day_search,recent_search,how='left',on='기업명')
merged

,청약일검색량,최근일주일검색량
기업명,,
ING생명보험,57.449205,100.000000
LG에너지솔루션,85.464685,66.502801
NH올원리츠,57.859765,63.817326
SK리츠,63.899235,59.559797
SK바이오사이언스,74.984995,64.596631
...,...,...
현대중공업,73.482235,63.028760
호전실업,60.733840,64.110640
화승엔터프라이즈,61.744120,81.536010


In [275]:
import math
score_list = []
for i in range(len(merged['청약일검색량'])):
    sub_search_amt = round(merged['청약일검색량'][i],3)
    weekly_search_amt = round(merged['최근일주일검색량'][i],3)
    score = math.log10(sub_search_amt) - math.log10(weekly_search_amt)
    score_list.append(score)
merged = merged.assign(비정상검색량지수 = score_list)
merged

,청약일검색량,최근일주일검색량,비정상검색량지수
기업명,,,
ING생명보험,57.449205,100.000000,-0.240718
LG에너지솔루션,85.464685,66.502801,0.108947
NH올원리츠,57.859765,63.817326,-0.042558
SK리츠,63.899235,59.559797,0.030539
SK바이오사이언스,74.984995,64.596631,0.064762
...,...,...,...
현대중공업,73.482235,63.028760,0.066641
호전실업,60.733840,64.110640,-0.023501
화승엔터프라이즈,61.744120,81.536010,-0.120755
